# sumaryzacja tekstu

In [1]:
!pip install "datasets<3.0.0"


## importowanie datesetu

In [2]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [3]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [4]:
!pip install bert-extractive-summarizer

In [5]:
from summarizer import Summarizer
model = Summarizer()

In [6]:
!pip install pytorch-ignite
!pip install torchmetrics

In [7]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [8]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [9]:
num_workers = 1
batch_size = 1

In [10]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [22]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        do_sample=True,
        top_k=0,
        temperature=0.6,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    return bertscore(predictions, batch["summary"])

In [12]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [13]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [14]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [15]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt)
    return loss

In [ ]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=2, bert=True, tokenizer=None)
print(loss)



Doc: The massive crater is filled with debris, household items and vehicles.
The hole is just four miles 
Prediction: The massive crater is filled with debris, household items and vehicles. The hole is just four miles from the Arena das Dunas stadium, which is set to host the Group D match between Italy and Uruguay on Tuesday.
Target: Houses in a favela, near one of Brazil's World Cup stadiums, have been evacuated after this huge sinkhole opened up.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: Police said that someone opened fire from a passing car and witnesses spoke of up to a dozen shots b
Prediction: Police said that someone opened fire from a passing car and witnesses spoke of up to a dozen shots being fired close to the main police station. The shots were fired in Friedrich-Stoltze Square, a busy area of bars and cafes, as local people enjoyed the Ascension Day holiday in the sunshine.
Target: At least three people were wounded, two seriously, when shots were fired in the German city of Frankfurt on Thursday.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: The car tried to crash through the barriers at Place de la Concorde, and police opened fire as it sp
Prediction: The car tried to crash through the barriers at Place de la Concorde, and police opened fire as it sped away. A third person, a young woman in hospital with bullet wounds, is also thought to have been in the car.
Target: Paris officials, investigating why a car was driven through the security barriers ahead of the Tour de France on Sunday morning, are holding two people who handed themselves in overnight.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, fa
Prediction: Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, families would have to ration their food and had little communication in their homes. Luxuries like chocolate and fruit were very difficult to find and families had to grow their own food to survive.
Target: Martin went to the German Occupation Museum to see what life was like for a family living on Guernsey in World War II.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

KeyboardInterrupt: 

In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

The following layers were not sharded: transformer.h.*.mlp.c_fc.weight, transformer.ln_f.bias, lm_head.weight, transformer.h.*.mlp.c_proj.weight, transformer.h.*.mlp.c_proj.bias, transformer.h.*.attn.c_attn.bias, transformer.wte.weight, transformer.wpe.weight, transformer.ln_f.weight, transformer.h.*.ln_*.weight, transformer.h.*.attn.c_attn.weight, transformer.h.*.attn.c_proj.weight, transformer.h.*.mlp.c_fc.bias, transformer.h.*.ln_*.bias, transformer.h.*.attn.c_proj.bias


In [20]:
def generate_prompt(tokenizer, size):
  prompt = f'Summarize texts:\n'
  shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
  for batch in shots_loader:
    for i in range(len(batch['document'])):
      prompt += f'{i+1}. {batch['document'][i]} -> {batch['summary'][i]}'
      prompt += tokenizer.eos_token

  prompt += "Summarize texts:\n"
  return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

Summarize texts:
1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B: -> Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>2. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors

In [23]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=10, bert=False, tokenizer=tokenizer, prompt = prompt))


Summarize texts:
1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B: -> Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>2. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: More than 20 firefighters were sent to the Westmorland Shopping Centre in Kendal just after 08:00 BS
Prediction: 
"We're calling on everyone to remain indoors," he said.
In a statement, the South African fire service said: "A cordon was put in place around the split-level complex, which houses about 30 shops and an indoor market
Target: A Cumbria shopping centre had to be evacuated after a fire started in a shop storeroom.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Lawless, 31, was a free agent, having been released by Luton Town when his contract at Kenilworth Ro
Prediction: 
Target: Yeovil Town have signed midfielder Alex Lawless on a one-year deal.


...

Doc: 23 December 2016 Last updated at 01:13 GMT
But much of India's milk is collected from small farms ac
Prediction: 
"We have got a lot of cows, in fact, we have about 50 cows. So we have 20 to 25 cows, but we have got some cows that are not that good. We have given them a fresh milk so that they can
Target: India is a huge consumer of milk - it's used to make yoghurt, cheese and a wide range of Indian sweets.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Top-flight sides Birmingham City and Manchester City will meet at the national stadium on Saturday, 
Prediction: 
The match will be broadcast on BBC Radio 1.
Target: The Women's FA Cup Final is set for a new competition-record crowd at Wembley, with 34,500 tickets already sold eight days before the 2017 final.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.7878, 0.2803, 0.3525, 0.7438, 0.7136, 0.7832, 0.7592, 0.6876, 0.7345,
        0.8099]), 'recall': tensor([0.8494, 0.0862, 0.1456, 0.8106, 0.7571, 0.8070, 0.7255, 0.6423, 0.7745,
        0.7602]), 'f1': tensor([0.8175, 0.1319, 0.2060, 0.7757, 0.7347, 0.7949, 0.7420, 0.6642, 0.7539,
        0.7842])}
(tensor(0.6652), tensor(0.6358), tensor(0.6405))


In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: lm_head.weight, model.layers.*.post_attention_layernorm.weight, model.embed_tokens.weight, model.layers.*.input_layernorm.weight, model.norm.weight, model.layers.*.self_attn.q_proj.bias, model.layers.*.self_attn.k_proj.weight, model.layers.*.self_attn.v_proj.weight


In [26]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=2, bert=False, tokenizer=tokenizer3, prompt=prompt3))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: The party is not happy that it was only allocated 25 seconds of airtime which was aired a month befo
Prediction:  -> The Workers Party's plans to meet with both the BBC and the Electoral Commission after polling next Thursday are set to be controversial. The party is not happy that it was only allocated 25 seconds of airtime which was aired a month before polling
Target: Members of the Workers Party have held a protest outside the BBC's headquarters in Belfast to highlight what they believe is a lack of coverage given to smaller parties during the Assembly election campaign.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

Doc: A 56-year-old man had to be placed in a medically-induced coma after he was attacked on Davies Row i
Prediction:  He said: "He has a history of violent behaviour and we are concerned about his mental health."
A 26-year-old man was also arrested on Tuesday and charged with attempting to make an unauthorised entry and the use of a false passport
Target: A man has been charged with attempted murder following an assault in Denny.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.7903, 0.7845]), 'recall': tensor([0.8275, 0.8281]), 'f1': tensor([0.8085, 0.8057])}
(tensor(0.7874), tensor(0.8278), tensor(0.8071))
